Code die checkt of wij wav-files correct wegschrijven naar tf-records.


[link to original code](https://github.com/tensorflow/models/tree/master/research/audioset)

In [ ]:
## Import packages
import os
import csv
import random
import numpy as np
import pandas as pd
from scipy.io import wavfile

import tensorflow as tf
from tensorflow import gfile
from tensorflow import logging

import vggish_input
import vggish_postprocess
import vggish_params
import vggish_slim

In [ ]:
flags = tf.app.flags
FLAGS = flags.FLAGS
tf.app.flags.DEFINE_string('f', '', 'kernel')

flags.DEFINE_string(
    'audio_embedding_feature_name', 'audio_embedding',
    'Path to the VGGish checkpoint file.')

## De volgende bestanden komen uit het eerdere mapje 'models'. Nu heb ik ze in yt8m gezet.
## Deze zijn nodig om de embedding layer te maken
flags.DEFINE_string(
    'pca_params', 'yt8m/vggish_pca_params.npz',
    'Path to the VGGish PCA parameters file.')

flags.DEFINE_string(
    'checkpoint', 'yt8m/vggish_model.ckpt',
    'Path to the VGGish checkpoint file.')

flags.DEFINE_string(
    'yt_checkpoint', 'yt8m/youtube_model.ckpt',
    'Path to the VGGish checkpoint file.')

In [22]:
filename = 'test_embedding/_D.tfrecord'    
record_iterator = tf.python_io.tf_record_iterator(filename)

## Met break: check alleen het eerste geluidsfragment uit de tf-record
embedding = None
    
for string_record in record_iterator:

    ## Take one example
    example = tf.train.SequenceExample()
    example.ParseFromString(string_record)

    embedding = example.feature_lists.feature_list['audio_embedding']

    break;

In [23]:
example

context {
  feature {
    key: "end_time_seconds"
    value {
      float_list {
        value: 150.0
      }
    }
  }
  feature {
    key: "labels"
    value {
      int64_list {
        value: 161
        value: 162
        value: 164
        value: 165
        value: 166
        value: 167
        value: 168
      }
    }
  }
  feature {
    key: "start_time_seconds"
    value {
      float_list {
        value: 140.0
      }
    }
  }
  feature {
    key: "video_id"
    value {
      bytes_list {
        value: "_DHMdtRRJzE"
      }
    }
  }
}
feature_lists {
  feature_list {
    key: "audio_embedding"
    value {
      feature {
        bytes_list {
          value: "\330w\026\201\223rD\325\000U\000\234\312u\253\223\303\210\303\036\230/\303\356\301nM:~\205\207}\377\265\204~\303S}\247Wo`6@\263\333\230\227\257g*\237~z\212*\377~\021\201X\207B\335\302\324\227\213\234^\202O\211c\203<iZ\247S\202;\371\272}c\022\267k\377\207\036m\210\206\232\324k\000em\266^\2256X\341\201\333dt>\235d\243

In [11]:
#byte_array = b'\xd8w\x16\x81\x93rD\xd5\x00U\x00\x9c\xcau\xab\x93\xc3\x88\xc3\x1e\x98/\xc3\xee\xc1nM:~\x85\x87}\xff\xb5\x84~\xc3S}\xa7Wo`6@\xb3\xdb\x98\x97\xafg*\x9f~z\x8a*\xff~\x11\x81X\x87B\xdd\xc2\xd4\x97\x8b\x9c^\x82O\x89c\x83<iZ\xa7S\x82;\xf9\xba}c\x12\xb7k\xff\x87\x1em\x88\x86\x9a\xd4k\x00em\xb6^\x956X\xe1\x81\xdbdt>\x9dd\xa3\xf5\xcb\xe6Z-\xc5!\xb2\xf1\xe6\x80\xe5'
print(embedding.feature[0].bytes_list.value)

[b"\\\x8fy\xdc\x7ff\xcf$\xff\xc6\x00\x00\x1c\xd1\x8e]\x17\xc6\xf1Z\xda'\xe0\xc3\xffw\x87\xd7\x00}RK\x97\xb4\xf0aA\xb3\x00\xa1s\xd5\xff\xa1[y\xb9#\x80w\x85\xa7\x00\x00\xff\x00\x9e,Ic\xd4s\xb7X\x86\x14AxX\x91\x10\xb8\x00\xcer\xe75\x1cU\x00\x8f\x1e\xff\x00\xf5\x9e\xdb\xfae\x00\xbe\x85\x004\xff\xf3-s\xff:\xc5`\x9e%\xff7DX\x8d\xb7\xff\x00\xd33\xf5\x00/\xc9\xe0I,\xff\xff\xff(\x1b\x12\x89"]


In [24]:
## Haal nu het wav-bestandje op dat van Youtube is gehaald
wavfile_path = 'test_embedding/youtube_D.wav'

In [25]:
## Deze code is van inference.py gehaald.
## Op het einde wordt er een nieuwe sequence example gemaakt

## Function that takes examples from wav-file as input and returns a sequence example
def getSequenceExample(examples_batch, labels, video_id=[b'_DHMdtRRJzE']):
    with tf.Graph().as_default(), tf.Session() as sess:

        # Define the model: load the checkpoint and locate input and output tensors
        # Input: [batch_size, num_frames, num_bands] 
        # where [num_frames, num_bands] represents log-mel-scale spectrogram
        # Output: embeddings
        vggish_slim.define_vggish_slim(training=False)
        vggish_slim.load_vggish_slim_checkpoint(sess, FLAGS.checkpoint)

        pca_params = np.load(FLAGS.pca_params)
        pca_matrix = pca_params[vggish_params.PCA_EIGEN_VECTORS_NAME]
        pca_means = pca_params[vggish_params.PCA_MEANS_NAME].reshape(-1, 1)

        features_tensor = sess.graph.get_tensor_by_name(
            vggish_params.VGGISH_INPUT_TENSOR_NAME)
        embedding_tensor = sess.graph.get_tensor_by_name(
            vggish_params.VGGISH_OUTPUT_TENSOR_NAME)
        vggish_slim.load_youtube_model(sess, FLAGS.yt_checkpoint)

        # Run inference and postprocessing
        [embedding_batch] = sess.run([embedding_tensor],
                                     feed_dict={features_tensor: examples_batch})

        postprocessed_batch = np.dot(
                pca_matrix, (embedding_batch.T - pca_means)
            ).T
        #print(postprocessed_batch)

        num_frames = np.minimum(postprocessed_batch.shape[0], vggish_params.MAX_FRAMES)
        data = vggish_postprocess.resize(postprocessed_batch, 0, vggish_params.MAX_FRAMES)
        data = np.expand_dims(data, 0)
        num_frames = np.expand_dims(num_frames, 0)

        input_tensor = sess.graph.get_collection("input_batch_raw")[0]
        num_frames_tensor = sess.graph.get_collection("num_frames")[0]

        label_feat = tf.train.Feature(int64_list=tf.train.Int64List(value=labels))
        videoid_feat = tf.train.Feature(bytes_list=tf.train.BytesList(value=video_id))
 
        seq_example = tf.train.SequenceExample(
            context = tf.train.Features(feature={"labels": label_feat, "video_id": videoid_feat}),
            feature_lists = tf.train.FeatureLists(
                feature_list={
                    FLAGS.audio_embedding_feature_name:
                        tf.train.FeatureList(
                            feature=[
                                tf.train.Feature(
                                    bytes_list=tf.train.BytesList(
                                        value=[embedding.tobytes()]))
                                for embedding in postprocessed_batch
                            ]
                        )
                }
            )
        )
    return(seq_example)

In [26]:
## Maak eerst de examples
examples_batch = vggish_input.wavfile_to_examples(wavfile_path)
print("Examples shape: " + str(examples_batch.shape))

## Maak nu de embedding (met functie hierboven), pas PCA toe en maak er bytes van
seq_example = getSequenceExample(examples_batch, [161])

Examples shape: (10, 96, 64)
INFO:tensorflow:Restoring parameters from yt8m/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from yt8m/youtube_model.ckpt


In [31]:
print(seq_example)

context {
  feature {
    key: "labels"
    value {
      int64_list {
        value: 161
      }
    }
  }
  feature {
    key: "video_id"
    value {
      bytes_list {
        value: "_DHMdtRRJzE"
      }
    }
  }
}
feature_lists {
  feature_list {
    key: "audio_embedding"
    value {
      feature {
        bytes_list {
          value: "\013C\357\310}\340\352?\017\027\321\342\007\245\373\277\216G\300\330\300\321\210?s\227\222\220\247\326\321\277\006\320q\254\362\201\372?\341\257nxe\310\361\277z\250s0\232i\351\277~1rc\272,\303\277\376\323_\357d\345\367\277HRSP\356\252\354?\271M[\023\346\216\354\277\320;\3377\253\003\342\277\207\340\177\324C\220\351?\316\253\242\355\377\'\321\277\263NrM>o\314\277\222jq\215\236s\360\277\251b\312^\367x\344?\0248\177s=\034\342\277m\264\261z\006\252\356?\276\222\336\272pE\273\277i\024\345 \206\353\361\277A\200\344\203\212\247\355?H\327hC/*\320\277\000*\007\242>`\253\277\230\316\033\034\352J\275?\036\277\204\220`\030\324?5\301\261\001\225o\355?\224\24

In [35]:
seq_example.feature_lists.feature_list['audio_embedding'].feature[0].bytes_list.value

[b'\x0bC\xef\xc8}\xe0\xea?\x0f\x17\xd1\xe2\x07\xa5\xfb\xbf\x8eG\xc0\xd8\xc0\xd1\x88?s\x97\x92\x90\xa7\xd6\xd1\xbf\x06\xd0q\xac\xf2\x81\xfa?\xe1\xafnxe\xc8\xf1\xbfz\xa8s0\x9ai\xe9\xbf~1rc\xba,\xc3\xbf\xfe\xd3_\xefd\xe5\xf7\xbfHRSP\xee\xaa\xec?\xb9M[\x13\xe6\x8e\xec\xbf\xd0;\xdf7\xab\x03\xe2\xbf\x87\xe0\x7f\xd4C\x90\xe9?\xce\xab\xa2\xed\xff\'\xd1\xbf\xb3NrM>o\xcc\xbf\x92jq\x8d\x9es\xf0\xbf\xa9b\xca^\xf7x\xe4?\x148\x7fs=\x1c\xe2\xbfm\xb4\xb1z\x06\xaa\xee?\xbe\x92\xde\xbapE\xbb\xbfi\x14\xe5 \x86\xeb\xf1\xbfA\x80\xe4\x83\x8a\xa7\xed?H\xd7hC/*\xd0\xbf\x00*\x07\xa2>`\xab\xbf\x98\xce\x1b\x1c\xeaJ\xbd?\x1e\xbf\x84\x90`\x18\xd4?5\xc1\xb1\x01\x95o\xed?\x94\xa4~\xd3\xdcH\xd1?R\xc1k\xc6\xcd\x89\xe7?A\x8f\x17\xe6U\xc3\xe6?\xc0\xf2\x9cP\xech\xdf?M\xf5\x884El\xf3\xbf\xa0y\x8b\xd1?\xa9\xed?\x8a\x9d\x8d\xf47\xec\xd4?\xa0\xadK\x077\xcb\xda?\x1a\x07\xe8\xbd\xae\x80\xd7?\xb8\xa0\x8d\xfd@\xd9\xaa?<=g\xd0\'\x80\xe3\xbf\x05\xdaX\xe5\x84\xad\xf0\xbfQ#n\xaf\xffU\xf0\xbf\x06.J\xf1h\xad\xda?\xe3\x12\x82\x7f:t\xe5

In [ ]:
# Vergelijk met de eerste:
# print(embedding.feature[0].bytes_list.value)